In [ ]:
# Testing Cell
from aviary.interface.cmd_entry_points import _command_map
from aviary.utils.doctape import glue_variable

draw_mission = 'draw_mission'
_command_map[draw_mission]
glue_variable(draw_mission, 'aviary ' + draw_mission, md_code=True)

# Mission Optimization with Many Phases for a Commercial Aircraft

So far within these example docs we have been building up the complexity of our coupled aircraft-mission design problem.
In [the simple mission example](simple_mission_example), we flew the aircraft in straight line phases.
In [the more advanced mission example](more_advanced_example), we allowed the optimizer to find the optimal Mach profiles for phases.

In this example, we will build on the prior examples by adding more phases to the mission.
This will allow us to model more complex missions, such as a commercial aircraft flying a long-haul route with multiple cruise segments, intermediary climb segments, and a cruise-climb segment.

## Problem Formulation

We use the {glue:md}`draw_mission` GUI as shown below:

![multiple_phases_gui](images/multiple_phases_gui.png)

This results in the following `phase_info` dictionary:

In [ ]:
phase_info = {
    'pre_mission': {'include_takeoff': False, 'optimize_mass': True},
    'climb_1': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.2, 'unitless'),
            'mach_final': (0.72, 'unitless'),
            'mach_bounds': ((0.18, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (0.0, 'ft'),
            'altitude_final': (31000.0, 'ft'),
            'altitude_bounds': ((0.0, 31500.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'time_initial_bounds': ((0.0, 0.0), 'min'),
            'time_duration_bounds': ((25.5, 76.5), 'min'),
        },
        'initial_guesses': {'time': ([0, 51], 'min')},
    },
    'cruise_1': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.72, 'unitless'),
            'mach_final': (0.72, 'unitless'),
            'mach_bounds': ((0.7, 0.74), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (31000.0, 'ft'),
            'altitude_final': (31000.0, 'ft'),
            'altitude_bounds': ((30500.0, 31500.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((25.5, 76.5), 'min'),
            'time_duration_bounds': ((23.5, 70.5), 'min'),
        },
        'initial_guesses': {'time': ([51, 47], 'min')},
    },
    'climb_2': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.72, 'unitless'),
            'mach_final': (0.74, 'unitless'),
            'mach_bounds': ((0.7, 0.76), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (31000.0, 'ft'),
            'altitude_final': (33000.0, 'ft'),
            'altitude_bounds': ((30500.0, 33500.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((49.0, 147.0), 'min'),
            'time_duration_bounds': ((5.0, 15.0), 'min'),
        },
        'initial_guesses': {'time': ([98, 10], 'min')},
    },
    'cruise_2': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.74, 'unitless'),
            'mach_final': (0.74, 'unitless'),
            'mach_bounds': ((0.72, 0.76), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (33000.0, 'ft'),
            'altitude_final': (33000.0, 'ft'),
            'altitude_bounds': ((32500.0, 33500.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((54.0, 162.0), 'min'),
            'time_duration_bounds': ((24.0, 72.0), 'min'),
        },
        'initial_guesses': {'time': ([108, 48], 'min')},
    },
    'climb_3': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.74, 'unitless'),
            'mach_final': (0.76, 'unitless'),
            'mach_bounds': ((0.72, 0.78), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (33000.0, 'ft'),
            'altitude_final': (34500.0, 'ft'),
            'altitude_bounds': ((32500.0, 35000.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((78.0, 234.0), 'min'),
            'time_duration_bounds': ((7.0, 21.0), 'min'),
        },
        'initial_guesses': {'time': ([156, 14], 'min')},
    },
    'climb_4': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.76, 'unitless'),
            'mach_final': (0.76, 'unitless'),
            'mach_bounds': ((0.74, 0.78), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (34500.0, 'ft'),
            'altitude_final': (36000.0, 'ft'),
            'altitude_bounds': ((34000.0, 36500.0), 'ft'),
            'throttle_enforcement': 'boundary_constraint',
            'time_initial_bounds': ((85.0, 255.0), 'min'),
            'time_duration_bounds': ((43.0, 129.0), 'min'),
        },
        'initial_guesses': {'time': ([170, 86], 'min')},
    },
    'descent_1': {
        'subsystem_options': {'aerodynamics': {'method': 'computed'}},
        'user_options': {
            'num_segments': 3,
            'order': 3,
            'mach_optimize': False,
            'mach_polynomial_order': 1,
            'mach_initial': (0.76, 'unitless'),
            'mach_final': (0.2, 'unitless'),
            'mach_bounds': ((0.18, 0.78), 'unitless'),
            'altitude_optimize': False,
            'altitude_polynomial_order': 1,
            'altitude_initial': (36000.0, 'ft'),
            'altitude_final': (500.0, 'ft'),
            'altitude_bounds': ((0.0, 36500.0), 'ft'),
            'throttle_enforcement': 'path_constraint',
            'time_initial_bounds': ((128.0, 384.0), 'min'),
            'time_duration_bounds': ((41.0, 123.0), 'min'),
        },
        'initial_guesses': {'time': ([256, 82], 'min')},
    },
    'post_mission': {
        'include_landing': False,
        'constrain_range': True,
        'target_range': (2393, 'nmi'),
    },
}

In [ ]:
# Testing Cell
from aviary.utils.doctape import glue_keys

# make all phase_info keys available for later use (e.g. mach_optimize and target_range below)
glue_keys(phase_info, display=False)

## Running Aviary with Updated Parameters

Let's now run Aviary with this multiphase mission and view the results.

In [ ]:
import aviary.api as av

prob = av.run_aviary(
    'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv',
    phase_info,
    optimizer='SLSQP',
    make_plots=True,
)

Now that we've run Aviary, we can look at the results.
Open up the automatically generated `traj_results_report.html` and scroll through it to visualize the results.

Here are the altitude and Mach profiles:

![Altitude and Mach Profiles](images/multiphase_flight_profile.png)


```{note}
Remember, we did not allow the optimizer to control either the Mach _or_ the altitude profiles. The optimizer varied the phase durations until the optimal mission profile was found.
```

## What Next?

The point of this doc page is to show that missions can be arbitrarily complex in terms of the number of phases and how they're classified.
If you want multiple climb, cruise, descent phases, that's absolutely something Aviary can handle.

There are a lot of options for how you could modify this example.
You could:

- enable the {glue:md}`mach_optimize` or {glue:md}`altitude_optimize` flags
- increase the {glue:md}`mach_polynomial_order` and {glue:md}`altitude_polynomial_order` so there's more flexibility in the optimized mission
- try different {glue:md}`target_range` values for the full mission range
- add an external subsystem to the phases

Playing around with a model and seeing how different settings affect the optimization and resulting aircraft design is always an enlightening experience.